In [11]:
%load_ext autoreload
%autoreload 2
from src.data_processing import load_data
import itertools
import string

from src.ngrams import *
from src.string_similarity import levenshtein
import operator
from src.data_processing import print_progress
from nltk import word_tokenize, pos_tag
from src.data_processing import load_cmu
from listener.ipatoarpabet import translate
from string import punctuation

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
task1, task2, task3, min_pairs, strings, pun_strings = load_data()
cmu = load_cmu()

In [16]:
for p in task1:
    if not p['pun']:
        print(' '.join(p['words']))

No , baby oil does NOT come from squeezing dead babies .
If you can ' t be good , be careful .
And Adam asked , " What ' s a Headache ? "
The best things come in small packages .
A good mind possesses a kingdom .
There ' s no peace for the wicked .
In an attempt to kill a fly , I drove into a pole .
Features should be discovered , not documented !
" I drank WHAT ? ! ? ! " - Socrates
And Homer saith unto them , " D ' OH ! " - - Bart 17 : 3
Desperate times call for desperate measures .
There are no answers , only cross - references .
Guns don ' t kill people . Bullets kill people .
I ' m spending a year dead for tax purposes .
Better an egg today than a hen tomorrow .
When in doubt , do nothing .
My name is Bond , James Bond of Borg . You will be shaken , not stirred .
Sufficient unto the day is the evil thereof .
Optimist : A Yugo owner with a trailer hitch !
Bloom where you are planted .
I ' d love to , but the President said he might drop in .
Rome wasn ' t built in a day .
Don ' t ta

In [ ]:
ngram_frequencies = json.load(open("data/ngram_frequencies.json"))
task2_frequencies = [n for i, n in enumerate(ngram_frequencies) if task1[i]['pun']]

In [ ]:
test_freqencies = [{'frequencies': t2} for t2 in task2_frequencies]
for i, t in enumerate(task2):
    test_freqencies[i]['text'] = t['words']
    test_freqencies[i]['target'] = t['target']

In [ ]:
with open("data/ngram_search_space.json") as f:
    ngram_search_space = json.load(f)
    

In [ ]:
search_space = []
frequencies = []
for i in range(len(ngram_search_space)):
    if 'Tom' not in test_freqencies[i]['text']:
        search_space.append(ngram_search_space[i])
        frequencies.append(test_freqencies[i])
        

In [ ]:
len(search_space)

In [10]:
def switch_score(distance, frequency_difference):
    return frequency_difference / ((distance + 1) ** 2)

In [15]:
def find_target(index):
    print(index)
    result = []
    search = search_space[index]
    for candidate, other_ngrams in search:
        try:
            if candidate[0] in cmu:
                cand_phone = cmu[candidate[0].lower()][0]
            else:
                cand_phone = translate(candidate[0])[0].split()
        except:
            continue
        
        size = len(other_ngrams)
        count = 0
        for ngram, freq in other_ngrams.items():
            print("{} / {}".format(count, size))
            count += 1
            switch_word = ngram.split()[1]
#             continue
            if any(w in punctuation for w in switch_word):
                continue
            try:
                if switch_word in cmu:
                    switch_phone = cmu[switch_word][0]
                else:
                    switch_phone = translate(switch_word)[0].split()
            except:
                continue
            l_distance = levenshtein(cand_phone, switch_phone)
            if l_distance <= 2:
                result.append((switch_word, candidate[0], switch_score(l_distance, freq)))
                              
    return list(sorted(result, key=operator.itemgetter(2), reverse=True))

In [ ]:
from multiprocessing import Pool
p = Pool(4)
ranked_replacements = p.map(find_target, range(100))

21
0
7
14
15
1
16
8
17
2
18
3
4
22
23
5
24
25
26
27
28
29
30
19
20
35
31
32
33
34
42
43
44
45
46
47
48
49
6
36
9
10
11
12
13
37
38
39
56
40
57
58
59
41
63
64
70
71
72
73
60
61
62
65
74
66
67
68
69
77
78
79
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
75
76
80
81
82
83


In [4]:
from src.pun_algorithms import translate

In [5]:
translate('I used to be a word smith once a pun a time .', 'a pun')

AH/P/AH/N
Generating substitutions
 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 
Generating scores


[('upon', 44525.25),
 ('open', 48.5),
 ('spun', 11.75),
 ('upton', 11.0),
 ('pun', 0),
 ('oven', 0)]

In [57]:
def detect(sentence):
    # split context into unigrams and bigrams
    possible_indices = []
    context = sentence.split()
    for i in range(1, len(context)-1):
        freq = ngram_frequency(context[i-1:i+2])
        if freq < 250:
            possible_indices.append(i)
    
    scores = {}
    for i in possible_indices:
        if i > 0 and i < len(context) - 1:
            
            translation_scores = translate(sentence, context[i])
            if len(translation_scores) > 0:
                scores[context[i]] = translation_scores[0]
                print(context[i], translation_scores[0])
                
        if i > 0 and i < len(context) - 2:
            translation_scores = translate(sentence, ' '.join(context[i:i+2]))
            if len(translation_scores) > 0:
                scores[' '.join(context[i:i+2])] = translation_scores[0]
    return sorted(scores.items(), key=operator.itemgetter(1), reverse = True)

In [17]:
detect('A nudist has no reason to fear a pickpocket .')

tokenizing
{1: [1, 2]}
N/UW/D/IH/S/T
Generating substitutions
 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 
Generating scores
HH/AE/Z
Generating substitutions
 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 
Generating scores


{'has': [('hatch', 0), ('haves', 0), ('had', 0)],
 'nudist': [('buddhist', 0), ('rudest', 0), ('nuanced', 0)]}

In [60]:
test = [('word', ('wordy', -48.0)),
 ('once', ('whats', 0)),
 ('a pun', ('upon', 44525.25)),
 ('a time', ('time', -15.5)),
 ('a', ('the', -962.5)),
 ('smith once', ('smithson', 0)),
 ('smith', ('smear', 0)),
 ('pun', ('run', 1082.25)),
 ('pun a', ('punch', -17.75)),
 ('once a', ('once', 0))]

In [8]:
ngram_frequency('once upon a time'.split())

158410

In [38]:
res = detect('I used to be a word smith once a pun a time .', 2)

{1: [5, 6, 7, 9], 2: [5, 6, 7, 8, 9, 10]}
W/ER/D
Generating substitutions
 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 
Generating scores
S/M/IH/TH
Generating substitutions
 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 
Generating scores
W/AH/N/S
Generating substitutions
 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 
Generating scores
P/AH/N
Generating substitutions
 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 
Generating scores
W/ER/D/S/M/IH/TH
Generating substitutions
Generating scores
S/M/IH/TH/W/AH/N/S
Generating substitutions
 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 
Generating scores
W/AH/N/S/AH
Generating substitutions
 |████████████████████████████████

In [39]:
res

{'a pun': [('upon', 44525.25), ('open', 48.5), ('spun', 11.75)],
 'a time': [('attire', -15.5), ('time', -15.5)],
 'once': [('whats', 0), ('wuss', 0), ("what's", 0)],
 'once a': [('once', 0)],
 'pun': [('run', 1082.25), ('one', 619.0), ('ton', 602.75)],
 'pun a': [('punch', -17.75), ('punt', -29.75), ('pun', -29.75)],
 'smith': [('scythe', 0), ('myth', 0), ('smit', 0)],
 'smith once': [('smithson', 0)],
 'word': [('wood', -48.0), ('work', -48.0), ('heard', -48.0)]}

In [41]:
list(sorted(res.items(), key=lambda x: x[1][0][1], reverse=True))

[('a pun', [('upon', 44525.25), ('open', 48.5), ('spun', 11.75)]),
 ('pun', [('run', 1082.25), ('one', 619.0), ('ton', 602.75)]),
 ('smith once', [('smithson', 0)]),
 ('smith', [('scythe', 0), ('myth', 0), ('smit', 0)]),
 ('once', [('whats', 0), ('wuss', 0), ("what's", 0)]),
 ('once a', [('once', 0)]),
 ('a time', [('attire', -15.5), ('time', -15.5)]),
 ('pun a', [('punch', -17.75), ('punt', -29.75), ('pun', -29.75)]),
 ('word', [('wood', -48.0), ('work', -48.0), ('heard', -48.0)])]

In [7]:
test = [('word', ('wood', -48.0)),
 ('once', ('whats', 0)),
 ('a pun', ('upon', 44525.25)),
 ('a', ('the', -962.5)),
 ('smith once', ('smithson', 0)),
 ('smith', ('scythe', 0)),
 ('pun', ('run', 1082.25)),
 ('pun a', ('punch', -17.75)),
 ('once a', ('once', 0)),
 ('a time', ('attire', -15.5))]

In [10]:
list(sorted(test, key=lambda x: x[1][1], reverse=True))

[('a pun', ('upon', 44525.25)),
 ('pun', ('run', 1082.25)),
 ('once', ('whats', 0)),
 ('smith once', ('smithson', 0)),
 ('smith', ('scythe', 0)),
 ('once a', ('once', 0)),
 ('a time', ('attire', -15.5)),
 ('pun a', ('punch', -17.75)),
 ('word', ('wood', -48.0)),
 ('a', ('the', -962.5))]

In [17]:
from nltk import pos_tag

In [21]:
pos_tag("I was a word smith once a pun a time".split(), tagset='universal')

[('I', 'PRON'),
 ('was', 'VERB'),
 ('a', 'DET'),
 ('word', 'NOUN'),
 ('smith', 'NOUN'),
 ('once', 'ADV'),
 ('a', 'DET'),
 ('pun', 'NOUN'),
 ('a', 'DET'),
 ('time', 'NOUN')]

In [3]:
from src.pun_algorithms import *

In [ ]:
detect('Home is were the fart is .')

{1: [1, 4, 5]}
IH/Z
Generating substitutions
 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 
Generating scores
F/AA/R/T
Generating substitutions
 |██████████████████████████████████████████████████████████████--------------------------------------| 62.5% 

In [7]:
detect("Those who hate classical music have my symphony .")

tokenizing
{1: [2, 3, 5, 7]}
HH/EY/T
Generating substitutions
 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 
Generating scores
K/L/AE/S/IH/K/AH/L
Generating substitutions
 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 
Generating scores
HH/AE/V
Generating substitutions
 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 
Generating scores
S/IH/M/F/AH/N/IY
Generating substitutions
 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 
Generating scores


{'classical': [('classics', -37.111111111111114),
  ('classic', -37.111111111111114),
  ('clavicle', -37.111111111111114)],
 'hate': [('haiti', -24.75), ('haze', -24.75), ('wait', -24.75)],
 'have': [('has', 14.25), ('heave', 0), ('ham', 0)],
 'symphony': [('sympathy', 1056.2222222222222),
  ('tiffany', -56.22222222222222),
  ('simmons', -69.88888888888889)]}

In [18]:
ngram_frequency("A nudist has".split())

0